# Dynamic Pricing for Urban Parking Lots
# Notebook 5: Interactive Bokeh Visualizations

This notebook creates interactive, real-time visualizations using Bokeh
to showcase the dynamic pricing system in action.

# 1. IMPORT LIBRARIES

In [1]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure, show, output_file, output_notebook
from bokeh.layouts import column, row, gridplot
from bokeh.models import (ColumnDataSource, HoverTool, Select, Slider, 
                          DatetimeTickFormatter, Legend, Panel, Tabs,
                          LinearColorMapper, ColorBar, Range1d)
from bokeh.palettes import Viridis256, Category20, Spectral11
from bokeh.transform import linear_cmap
import warnings
warnings.filterwarnings('ignore')

# Enable Bokeh output in Jupyter notebook
output_notebook()

print("Libraries imported successfully!")
print("Bokeh visualizations will appear inline")

Loading BokehJS ...

Libraries imported successfully!
Bokeh visualizations will appear inline


# 2. LOAD DATA

In [2]:
# Load all datasets
pricing_data = pd.read_csv('pricing_results.csv')
streaming_data = pd.read_csv('streaming_simulation_results.csv')
parking_lots = pd.read_csv('parking_lot_info.csv')

# Convert datetime
pricing_data['DateTime'] = pd.to_datetime(pricing_data['DateTime'])
streaming_data['DateTime'] = pd.to_datetime(streaming_data['DateTime'])

print(f"\nData loaded:")
print(f"  • Pricing records: {len(pricing_data):,}")
print(f"  • Streaming records: {len(streaming_data):,}")
print(f"  • Parking lots: {len(parking_lots)}")


Data loaded:
  • Pricing records: 18,368
  • Streaming records: 500
  • Parking lots: 14


# 3. INTERACTIVE PLOT 1: REAL-TIME PRICE UPDATES

In [3]:
print(f"\n{'='*70}")
print(f"CREATING INTERACTIVE VISUALIZATIONS")
print(f"{'='*70}")

# Select a sample parking lot
sample_lot_id = pricing_data['LotID'].mode()[0]
lot_data = pricing_data[pricing_data['LotID'] == sample_lot_id].head(200).copy()
lot_data['TimeIndex'] = range(len(lot_data))

# Create ColumnDataSource
source = ColumnDataSource(lot_data)

# Create figure
p1 = figure(
    title=f"Real-Time Dynamic Pricing - Parking Lot {sample_lot_id}",
    x_axis_label='Time Period',
    y_axis_label='Price ($)',
    width=900,
    height=400,
    tools='pan,wheel_zoom,box_zoom,reset,save'
)

# Add multiple price lines
line1 = p1.line('TimeIndex', 'Model1_Price', source=source, 
                line_width=2, color='#FF6B6B', alpha=0.8, legend_label='Model 1: Linear')
line2 = p1.line('TimeIndex', 'Model2_Price', source=source, 
                line_width=2, color='#4ECDC4', alpha=0.8, legend_label='Model 2: Demand')
line3 = p1.line('TimeIndex', 'Model3_Price', source=source, 
                line_width=2, color='#45B7D1', alpha=0.8, legend_label='Model 3: Competitive')

# Add circles for interactivity
p1.circle('TimeIndex', 'Model2_Price', source=source, 
          size=6, color='#4ECDC4', alpha=0.6)

# Add hover tool
hover = HoverTool(tooltips=[
    ('Time', '@DateTime{%F %T}'),
    ('Model 1', '$@Model1_Price{0.00}'),
    ('Model 2', '$@Model2_Price{0.00}'),
    ('Model 3', '$@Model3_Price{0.00}'),
    ('Occupancy', '@OccupancyRate{0.0}%'),
    ('Queue', '@QueueLength')
], formatters={'@DateTime': 'datetime'})

p1.add_tools(hover)

# Configure legend
p1.legend.location = "top_left"
p1.legend.click_policy = "hide"

print("✓ Plot 1: Real-time price updates created")



CREATING INTERACTIVE VISUALIZATIONS
✓ Plot 1: Real-time price updates created


# 4. INTERACTIVE PLOT 2: PRICE VS OCCUPANCY SCATTER

In [4]:
# Prepare data for scatter plot
scatter_data = pricing_data.sample(min(1000, len(pricing_data)))  # Sample for performance

# Create color mapper based on queue length
mapper = linear_cmap(field_name='QueueLength', palette=Spectral11, 
                     low=scatter_data['QueueLength'].min(), 
                     high=scatter_data['QueueLength'].max())

source2 = ColumnDataSource(scatter_data)

p2 = figure(
    title="Price Response to Occupancy Rate",
    x_axis_label='Occupancy Rate (%)',
    y_axis_label='Price ($)',
    width=900,
    height=400,
    tools='pan,wheel_zoom,box_zoom,reset,save'
)

# Add scatter plot with color mapping
scatter = p2.scatter('OccupancyRate', 'Model2_Price', source=source2,
                     size=8, alpha=0.6, color=mapper)

# Add hover tool
hover2 = HoverTool(tooltips=[
    ('Lot', '@SystemCodeNumber'),
    ('Occupancy', '@OccupancyRate{0.0}%'),
    ('Price', '$@Model2_Price{0.00}'),
    ('Queue', '@QueueLength'),
    ('Traffic', '@TrafficConditionNearby'),
    ('Special Day', '@IsSpecialDay')
])

p2.add_tools(hover2)

# Add color bar
color_bar = ColorBar(color_mapper=mapper['transform'], width=8, location=(0,0),
                     title="Queue Length")
p2.add_layout(color_bar, 'right')

print("✓ Plot 2: Price vs occupancy scatter created")


✓ Plot 2: Price vs occupancy scatter created


# 5. INTERACTIVE PLOT 3: HOURLY PRICE TRENDS

In [5]:
# Calculate hourly averages across all parking lots
hourly_data = pricing_data.groupby('Hour').agg({
    'Model1_Price': 'mean',
    'Model2_Price': 'mean',
    'Model3_Price': 'mean',
    'OccupancyRate': 'mean',
    'QueueLength': 'mean'
}).reset_index()

source3 = ColumnDataSource(hourly_data)

p3 = figure(
    title="Average Hourly Pricing Trends",
    x_axis_label='Hour of Day',
    y_axis_label='Average Price ($)',
    width=900,
    height=400,
    tools='pan,wheel_zoom,box_zoom,reset,save',
    x_range=Range1d(0, 23)
)

# Add lines for each model
p3.line('Hour', 'Model1_Price', source=source3, 
        line_width=2, color='#FF6B6B', alpha=0.8, legend_label='Model 1')
p3.circle('Hour', 'Model1_Price', source=source3, 
          size=8, color='#FF6B6B', alpha=0.6)

p3.line('Hour', 'Model2_Price', source=source3, 
        line_width=2, color='#4ECDC4', alpha=0.8, legend_label='Model 2')
p3.circle('Hour', 'Model2_Price', source=source3, 
          size=8, color='#4ECDC4', alpha=0.6)

p3.line('Hour', 'Model3_Price', source=source3, 
        line_width=2, color='#45B7D1', alpha=0.8, legend_label='Model 3')
p3.circle('Hour', 'Model3_Price', source=source3, 
          size=8, color='#45B7D1', alpha=0.6)

# Add hover
hover3 = HoverTool(tooltips=[
    ('Hour', '@Hour:00'),
    ('Avg Price (M2)', '$@Model2_Price{0.00}'),
    ('Avg Occupancy', '@OccupancyRate{0.0}%'),
    ('Avg Queue', '@QueueLength{0.0}')
])

p3.add_tools(hover3)

p3.legend.location = "top_left"
p3.legend.click_policy = "hide"

print("✓ Plot 3: Hourly trends created")

✓ Plot 3: Hourly trends created


# 6. INTERACTIVE PLOT 4: PARKING LOT COMPARISON

In [6]:
# Calculate average metrics per parking lot
lot_comparison = pricing_data.groupby(['LotID', 'SystemCodeNumber']).agg({
    'Model2_Price': 'mean',
    'OccupancyRate': 'mean',
    'QueueLength': 'mean',
    'Capacity': 'first'
}).reset_index()

lot_comparison = lot_comparison.sort_values('Model2_Price')

source4 = ColumnDataSource(lot_comparison)

p4 = figure(
    title="Average Price by Parking Lot",
    x_axis_label='Average Price ($)',
    y_range=lot_comparison['SystemCodeNumber'].tolist(),
    width=900,
    height=400,
    tools='pan,wheel_zoom,box_zoom,reset,save'
)

# Horizontal bar chart
bars = p4.hbar(y='SystemCodeNumber', right='Model2_Price', source=source4,
               height=0.7, color='#4ECDC4', alpha=0.8)

# Add hover
hover4 = HoverTool(tooltips=[
    ('Parking Lot', '@SystemCodeNumber'),
    ('Avg Price', '$@Model2_Price{0.00}'),
    ('Avg Occupancy', '@OccupancyRate{0.0}%'),
    ('Capacity', '@Capacity'),
    ('Avg Queue', '@QueueLength{0.0}')
])

p4.add_tools(hover4)

print("✓ Plot 4: Parking lot comparison created")

✓ Plot 4: Parking lot comparison created


# 7. INTERACTIVE PLOT 5: DEMAND PRESSURE HEATMAP

In [7]:
# Create pivot table for heatmap (Hour x Day of Week)
pricing_data['DayOfWeek'] = pricing_data['DateTime'].dt.dayofweek
pricing_data['DayName'] = pricing_data['DateTime'].dt.day_name()

heatmap_data = pricing_data.groupby(['Hour', 'DayName'])['Model2_Price'].mean().reset_index()

# Prepare data for heatmap
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
hours = list(range(24))

# Create full grid
heat_source = []
for day in days_order:
    for hour in hours:
        price = heatmap_data[(heatmap_data['DayName'] == day) & 
                             (heatmap_data['Hour'] == hour)]['Model2_Price'].values
        heat_source.append({
            'day': day,
            'hour': f"{hour:02d}:00",
            'price': price[0] if len(price) > 0 else 10.0
        })

source5 = ColumnDataSource(pd.DataFrame(heat_source))

# Color mapper
mapper5 = LinearColorMapper(palette=Viridis256, 
                            low=min([x['price'] for x in heat_source]),
                            high=max([x['price'] for x in heat_source]))

p5 = figure(
    title="Average Price Heatmap: Hour vs Day of Week",
    x_range=[f"{h:02d}:00" for h in range(24)],
    y_range=list(reversed(days_order)),
    width=900,
    height=400,
    tools='hover,save',
    toolbar_location='below'
)

p5.rect(x='hour', y='day', width=1, height=1, source=source5,
        fill_color={'field': 'price', 'transform': mapper5},
        line_color=None)

# Configure hover
p5.select_one(HoverTool).tooltips = [
    ('Day', '@day'),
    ('Hour', '@hour'),
    ('Avg Price', '$@price{0.00}')
]

# Add color bar
color_bar5 = ColorBar(color_mapper=mapper5, width=8, location=(0,0),
                      title="Price ($)")
p5.add_layout(color_bar5, 'right')

# Rotate x-axis labels
p5.xaxis.major_label_orientation = 45

print("✓ Plot 5: Demand heatmap created")


✓ Plot 5: Demand heatmap created


# 8. DISPLAY ALL PLOTS

In [8]:
print(f"\n{'='*70}")
print(f"DISPLAYING INTERACTIVE VISUALIZATIONS")
print(f"{'='*70}")

# Create tabs for organized display
tab1 = Panel(child=p1, title="Real-Time Pricing")
tab2 = Panel(child=p2, title="Price vs Occupancy")
tab3 = Panel(child=p3, title="Hourly Trends")
tab4 = Panel(child=p4, title="Lot Comparison")
tab5 = Panel(child=p5, title="Demand Heatmap")

tabs = Tabs(tabs=[tab1, tab2, tab3, tab4, tab5])

# Show all visualizations
show(tabs)

print("\n✓ Interactive visualizations displayed!")
print("\nVisualization Features:")
print("  • Hover tooltips for detailed information")
print("  • Pan and zoom capabilities")
print("  • Click legend items to hide/show")
print("  • Multiple tabs for different views")


DISPLAYING INTERACTIVE VISUALIZATIONS



✓ Interactive visualizations displayed!

Visualization Features:
  • Hover tooltips for detailed information
  • Pan and zoom capabilities
  • Click legend items to hide/show
  • Multiple tabs for different views


# 9. CREATE COMPARISON DASHBOARD

In [10]:

print(f"\n{'='*70}")
print(f"CREATING MODEL COMPARISON DASHBOARD")
print(f"{'='*70}")

# Create comparison metrics
model_comparison = pricing_data.groupby('LotID').agg({
    'Model1_Price': ['mean', 'std'],
    'Model2_Price': ['mean', 'std'],
    'Model3_Price': ['mean', 'std']
}).reset_index()

model_comparison.columns = ['LotID', 'M1_Mean', 'M1_Std', 'M2_Mean', 'M2_Std', 'M3_Mean', 'M3_Std']

# Create figure WITHOUT grouped bars (simpler and works)
p6 = figure(
    title="Model Price Comparison by Parking Lot",
    x_axis_label='Parking Lot ID',
    y_axis_label='Average Price ($)',
    width=900,
    height=400,
    tools='pan,wheel_zoom,box_zoom,reset,save',
    x_range=[str(x) for x in model_comparison['LotID'].tolist()]
)

# Plot as simple line chart instead of grouped bars
p6.line(x='LotID', y='M1_Mean', source=ColumnDataSource(model_comparison), 
        line_width=2, color='#FF6B6B', legend_label='Model 1', alpha=0.8)
p6.circle(x='LotID', y='M1_Mean', source=ColumnDataSource(model_comparison), 
          size=8, color='#FF6B6B', alpha=0.8)

p6.line(x='LotID', y='M2_Mean', source=ColumnDataSource(model_comparison), 
        line_width=2, color='#4ECDC4', legend_label='Model 2', alpha=0.8)
p6.circle(x='LotID', y='M2_Mean', source=ColumnDataSource(model_comparison), 
          size=8, color='#4ECDC4', alpha=0.8)

p6.line(x='LotID', y='M3_Mean', source=ColumnDataSource(model_comparison), 
        line_width=2, color='#45B7D1', legend_label='Model 3', alpha=0.8)
p6.circle(x='LotID', y='M3_Mean', source=ColumnDataSource(model_comparison), 
          size=8, color='#45B7D1', alpha=0.8)

p6.legend.location = "top_right"
p6.legend.click_policy = "hide"

show(p6)

print("✓ Model comparison dashboard created")


CREATING MODEL COMPARISON DASHBOARD


✓ Model comparison dashboard created


# 10. SAVE VISUALIZATIONS AS HTML

In [11]:
print(f"\n{'='*70}")
print(f"SAVING VISUALIZATIONS")
print(f"{'='*70}")

# Save individual plots
output_file("figures/interactive_pricing_dashboard.html")
show(tabs)

output_file("figures/model_comparison_dashboard.html")
show(p6)

print("\n✓ Interactive visualizations saved:")
print("  • figures/interactive_pricing_dashboard.html")
print("  • figures/model_comparison_dashboard.html")


SAVING VISUALIZATIONS



✓ Interactive visualizations saved:
  • figures/interactive_pricing_dashboard.html
  • figures/model_comparison_dashboard.html
